In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion

df = pd.read_csv(r"Credit.csv")
df = df.drop(columns=["Unnamed: 0"])
dfy = df["Rating"]
df.head(5)

In [ ]:
new_df = df.drop(columns=["Rating", "Limit", "Balance"])
numeric_columns2 = new_df.select_dtypes(include =[np.number]).columns.tolist()
cat_columns = new_df.select_dtypes(exclude=[np.number]).columns.tolist()
numeric_columns2
#cat_columns

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline, FeatureUnion

class MyLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = MultiLabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)

class DataframeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, atrib_names):
        self.atribute_names = atrib_names
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=0):
        return X[self.atribute_names].values
    

In [ ]:
num_pipeline = Pipeline([
    ('selector', DataframeSelector(numeric_columns2)),
    ('std_scaler', StandardScaler())
])
#categorial pipeline
cat_pipeline = Pipeline([
    ('selector', DataframeSelector(cat_columns)),
    ('label_binarizer', MyLabelBinarizer())
])
##union piplines
full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipline", cat_pipeline)
])

In [ ]:

X_data = new_df[numeric_columns2+cat_columns]
y_new = df["Limit"]

X_train, X_test, y_train, y_test = train_test_split(X_data, y_new, test_size=0.2, random_state=2)

X_train_trans = full_pipeline.fit_transform(X_train)
X_test_trans = full_pipeline.fit_transform(X_test)

X_train_trans[0]